In [1]:
!sudo apt install tesseract-ocr
!sudo apt-get install poppler-utils 
!pip install pytesseract pdf2image zero-shot-re neo4j

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# !git clone 'https://github.com/sudha-vijayakumar/BERN2_TigerGraph_BioMedical_KnowledgeGraph.git'

Cloning into 'BERN2_TigerGraph_BioMedical_KnowledgeGraph'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 24 (delta 4), reused 24 (delta 4), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [4]:
cd /content/drive/MyDrive/BERN2_TigerGraph_BioMedical_KnowledgeGraph/bioNLP 2/

/content/drive/MyDrive/BERN2_TigerGraph_BioMedical_KnowledgeGraph/bioNLP 2


## Parse pdf for sentences

In [5]:
import requests
import pdf2image
import pytesseract

pdf = '2111.09414.pdf'
doc = pdf2image.convert_from_path(pdf)


article = []
for page_number, page_data in enumerate(doc):
    txt = pytesseract.image_to_string(page_data).encode("utf-8")
    if page_number < 6:
      article.append(txt.decode("utf-8"))
article_txt = " ".join(article)


In [6]:
article_txt

"Developmental Status and Perspectives for Tissue Engineering in Urology.\n\nElcin Huseyn\nAzerbaijan State Oil and Industry\nUniversity, Baku, Azerbaijan\n\nelcin.huseyn@asoiu.edu.az\n\nAbstract: Tissue engineering technology and tissue cell-based stem cell research have made\ngreat strides in treating tissue and organ damage, correcting tissue and organ dysfunction, and\nreducing surgical complications. In the past, traditional methods have used biological substitutes for\ntissue repair materials, while tissue engineering technology has focused on merging sperm cells\nwith biological materials to form biological tissues with the same structure and function as their\nown tissues. The advantage is that tissue engineering technology can overcome donors. Material\nprocurement restrictions can effectively reduce complications. The aim of studying tissue\nengineering technology is to find sperm cells and suitable biological materials to replace the\noriginal biological functions of tissues

## Text preprocessing

In [7]:
import nltk
nltk.download('punkt')

def clean_text(text):
  """Remove section titles and figure descriptions from text"""
  clean = "\n".join([row for row in text.split("\n") if (len(row.split(" "))) > 3 and not (row.startswith("(a)"))
                    and not row.startswith("Figure")])
  return clean

text = article_txt.split("Abstract")[1]
ctext = clean_text(text)
sentences = nltk.tokenize.sent_tokenize(ctext)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import hashlib
import requests

#http://bern2.korea.ac.kr/documentation
def query_raw(text, url="http://bern2.korea.ac.kr/plain"):
    return requests.post(url, json={'text': text}).json()


entity_list = []
for s in sentences[:-1]:
  try:
    entity_list.append(query_raw(s))
  except:
    print('invalid sentence')
    
parsed_entities = []
for entities in entity_list:
  e = []
  if not entities.get('annotations'):
    parsed_entities.append({'text':entities['text'], 'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()})
    continue
  for entity in entities['annotations']:
    other_ids = [id for id in entity['id'] if not id.startswith("BERN")]
    entity_type = entity['obj']
    entity_name = entities['text'][entity['span']['begin']:entity['span']['end']]
    try:
      entity_id = [id for id in entity['id'] if id.startswith("BERN")][0]
    except IndexError:
      entity_id = entity_name
    e.append({'entity_id': entity_id, 'other_ids': other_ids, 'entity_type': entity_type, 'entity': entity_name})
  parsed_entities.append({'entities':e, 'text':entities['text'], 'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()})

invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence
invalid sentence


### Parse entities

In [ ]:
import json
with open('article.txt', 'w') as f:
    f.write(article_txt)


json_string = json.dumps(parsed_entities)
with open('parsed_entities.json', 'w') as outfile:
    outfile.write(json_string)

### Parse Relationships

In [ ]:
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor

model = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
relations = ['associated', 'interacts']
extractor = RelationExtractor(model, tokenizer, relations)

In [ ]:
import itertools

candidates = [s for s in parsed_entities if (s.get('entities')) and (len(s['entities']) > 1)]
predicted_rels = []
for c in candidates:
  combinations = itertools.combinations([{'name':x['entity'], 'id':x['entity_id']} for x in c['entities']], 2)
  for combination in list(combinations):
    try:
      ranked_rels = extractor.rank(text=c['text'].replace(",", " "), head=combination[0]['name'], tail=combination[1]['name'])
      
      if ranked_rels[0][1] > 0.85:
        predicted_rels.append({'head': combination[0]['id'], 'tail': combination[1]['id'], 'type':ranked_rels[0][0], 'source': c['text_sha256']})
    except:
      pass


In [ ]:
json_string = json.dumps(predicted_rels)
with open('predicted_rels.json', 'w') as outfile:
    outfile.write(json_string)